In [136]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.service import Service

In [137]:
import requests
from bs4 import BeautifulSoup
import json

In [138]:
import pandas as pd

In [139]:
SER = Service('/Users/jimmydelano/Downloads/chromedriver')
OP = webdriver.ChromeOptions()
URL = 'https://www.lennar.com/new-homes/alabama/huntsville'

In [242]:
driver = webdriver.Chrome(service=SER, options=OP)
driver.get(URL)
driver.maximize_window()
time.sleep(10)


try:
    driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div/button").click()
except:
    print("No cookies window. Continue...")
    
time.sleep(5)
# try:
#     driver.find_element(By.XPATH, '//*[@id="main"]/div/div[1]/div[2]/div/div[3]/div/button[1]').click()
# except:
#     print("Couldn't find list more homes...")


more_homes = True
while more_homes:
    try:
        load_more = driver.find_element(By.XPATH, '//button[normalize-space()="Load more homes"]')
        driver.execute_script("arguments[0].click();", load_more)
        time.sleep(5)
    except:
        print("No more homes to load... Scraping text")
        more_homes = False

keep_scraping = True
index = 1
data = []
while keep_scraping:
    if index == 2:
        index += 1
        continue
    
    try:
        result = driver.find_element(By.XPATH, 
                        f'//*[@id="main"]/div/div[1]/div[2]/div/div[2]/div/div[2]/div[{index}]').text.splitlines()
        
        ## data format error with extra column
        if len(result) != 7:
            assert result[0] == 'Flex Cash'
            result = result[1:]
            
        ## correct data format error with rearranging columns
        if result[0] == "Future release":
            result.insert(5, result.pop(1))
            
        data.append(result)
        index += 1
    except:
        print("Done Scraping info")
        keep_scraping = False

df = pd.DataFrame(data, columns = ["Availability", "Price", "Beds", "Baths", "Sqft", "Address", "Community"])

No more homes to load... Scraping text
Done Scraping info


In [243]:
df

,Availability,Price,Beds,Baths,Sqft,Address,Community
0,Move-in ready,"$318,600",3 bd,2 ba,"2,033 ft²","24212 Costello Dr, Athens, AL",Residence 2033 in Browns Crossing West
1,Move-in ready,"$327,900",3 bd,2 ba,"1,911 ft²","14960 Demarie Ln, Athens, AL",Residence 1911 in Browns Crossing West
2,Move-in ready,"$409,950",3 bd,2.5 ba,"2,369 ft²","310 Hay Rake Dr, Madison, AL",Farmhouse Unit 6 in Clift Farm
3,Move-in ready,"$409,950",3 bd,2.5 ba,"2,369 ft²","306 Hay Rake Dr, Madison, AL",Farmhouse Unit 6 in Clift Farm
4,Move-in ready,"$409,950",3 bd,2.5 ba,"2,369 ft²","308 Hay Rake Dr, Madison, AL",Farmhouse Unit 6 in Clift Farm
...,...,...,...,...,...,...,...
130,Future release,From $446K,4 bd,3 ba,"3,159 ft²",Residence 3159,Residence 3159 in The Retreat
131,Future release,From $418K,4 bd,3 ba,"2,338 ft²",Residence 2338,Residence 2338 in The Retreat
132,Future release,From $336K,3 bd,2 ba,"1,846 ft²",Residence 1846,Residence 1846 in The Retreat
133,Future release,From $331K,3 bd,2 ba,"1,696 ft²",Residence 1696,Residence 1696 in The Retreat


In [244]:
df.to_csv('./huntsville-al.csv')

In [141]:
## have the homes in the chrome window, now need to grab each one. 
## tried beautiful soup but it still just pulls 50 even from driver.page_source

In [142]:
# driver.page_source